# Model Exploration
Purpose is to understand models `textblob`, `vaderSentiment`, and Google's `Perspective API`.<br>
By: Jonathan Lo<br>
Date: 1/16/23

In [1]:
# Model Imports
from textblob import TextBlob
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from googleapiclient import discovery

# Overhead Imports
from json import load

In [2]:
# Loading secrets
SECRETS = load(open('../secrets.json', 'r'))

## Model Setup

In [3]:
# Create Model Collection class for testing

class ModelCollection:
    
    
    def __init__(self, gcp_api_key):
        self.perspective_client = discovery.build(
            "commentanalyzer",
            "v1alpha1",
            developerKey=gcp_api_key,
            discoveryServiceUrl="https://commentanalyzer.googleapis.com/$discovery/rest?version=v1alpha1",
            static_discovery=False
        )
        self.vader_analyzer = SentimentIntensityAnalyzer()
      
    
    def queryPerspective(self, text: str):
        """ Sends a request to Perspective API for classification.
        """
        analyze_request = {
            'comment': {'text': text},
            'requestedAttributes': {'TOXICITY': {}}
        }
        response = self.perspective_client.comments().analyze(body=analyze_request).execute()
        perspective_result = response['attributeScores']['TOXICITY']['spanScores'][0]['score']['value']
        return round(perspective_result,3)
    
    
    def queryTextBlobPolairty(self, text: str):
        """ Returns the polarity of the text using TextBlob.
        """
        return TextBlob(text).sentiment.polarity

    
    def queryTextBlobObjectivity(self, text: str):
        """ Returns the objectivity of the text using TextBlob.
        """
        return TextBlob(text).sentiment.subjectivity
    
    
    def queryVaderSentiment(self, text: str):
        """ Returns the sentiment score given by VaderSentiment.
        """
        res = self.vader_analyzer.polarity_scores(text)
        return res['compound']
    
    
    def queryAllModelsSingle(self, text: str):
        """ Querys all of the models.
        """
        result = {
            'perspectiveScore': self.queryPerspective(text),
            'tbPolairty': self.queryTextBlobPolairty(text),
            'tbObjectivity': self.queryTextBlobObjectivity(text),
            'vsScore': self.queryVaderSentiment(text)
        }
        return result
    
    
    def queryAllModelsBulk(self, text_items: list):
        """ Querys all of the models for each item in the sentence.
        """
        results = []
        for sentence in text_items:
            results.append(mc.queryAllModelsSingle(sentence))
        return results
    

## Sample Testing

In [4]:
# Initialize Model Collection and testing sentances
mc = ModelCollection(gcp_api_key=SECRETS['PerspectiveAPIKey'])
sentences = [
    'You must be the change you wish to see in the world.',
    'Spread love everywhere you go. Let no one ever come to you without leaving happier.',
    'There is No War in Ba Sing Se.',
    'Money is the root of all evil.',
    'He bought the watch from the mall next to the Walmart.',
    'She brought her car into the car wash made for washing to wash her car.',
    'She sells sea shells by the sea shore.'
]

In [5]:
# Querying using individual methods
results = []
for sentence in sentences:
    result = {
        'perspectiveScore': mc.queryPerspective(sentence),
        'tbPolairty': mc.queryTextBlobPolairty(sentence),
        'tbObjectivity': mc.queryTextBlobObjectivity(sentence),
        'vsScore': mc.queryVaderSentiment(sentence)
    }
    results.append(result)
    
results

[{'perspectiveScore': 0.041,
  'tbPolairty': 0.0,
  'tbObjectivity': 0.0,
  'vsScore': 0.4019},
 {'perspectiveScore': 0.05,
  'tbPolairty': 0.5,
  'tbObjectivity': 0.6,
  'vsScore': 0.0577},
 {'perspectiveScore': 0.038,
  'tbPolairty': 0.0,
  'tbObjectivity': 0.0,
  'vsScore': 0.4847},
 {'perspectiveScore': 0.136,
  'tbPolairty': -1.0,
  'tbObjectivity': 1.0,
  'vsScore': -0.6597},
 {'perspectiveScore': 0.014,
  'tbPolairty': 0.0,
  'tbObjectivity': 0.0,
  'vsScore': 0.0},
 {'perspectiveScore': 0.053,
  'tbPolairty': 0.0,
  'tbObjectivity': 0.0,
  'vsScore': 0.0},
 {'perspectiveScore': 0.046,
  'tbPolairty': 0.0,
  'tbObjectivity': 0.0,
  'vsScore': 0.0}]

In [6]:
# Querying using the all methods API
results = []
for sentence in sentences:
    results.append(mc.queryAllModelsSingle(sentence))
    
results

[{'perspectiveScore': 0.041,
  'tbPolairty': 0.0,
  'tbObjectivity': 0.0,
  'vsScore': 0.4019},
 {'perspectiveScore': 0.05,
  'tbPolairty': 0.5,
  'tbObjectivity': 0.6,
  'vsScore': 0.0577},
 {'perspectiveScore': 0.038,
  'tbPolairty': 0.0,
  'tbObjectivity': 0.0,
  'vsScore': 0.4847},
 {'perspectiveScore': 0.136,
  'tbPolairty': -1.0,
  'tbObjectivity': 1.0,
  'vsScore': -0.6597},
 {'perspectiveScore': 0.014,
  'tbPolairty': 0.0,
  'tbObjectivity': 0.0,
  'vsScore': 0.0},
 {'perspectiveScore': 0.053,
  'tbPolairty': 0.0,
  'tbObjectivity': 0.0,
  'vsScore': 0.0},
 {'perspectiveScore': 0.046,
  'tbPolairty': 0.0,
  'tbObjectivity': 0.0,
  'vsScore': 0.0}]

In [7]:
# Query using the bulk API
results = mc.queryAllModelsBulk(sentences)
results

[{'perspectiveScore': 0.041,
  'tbPolairty': 0.0,
  'tbObjectivity': 0.0,
  'vsScore': 0.4019},
 {'perspectiveScore': 0.05,
  'tbPolairty': 0.5,
  'tbObjectivity': 0.6,
  'vsScore': 0.0577},
 {'perspectiveScore': 0.038,
  'tbPolairty': 0.0,
  'tbObjectivity': 0.0,
  'vsScore': 0.4847},
 {'perspectiveScore': 0.136,
  'tbPolairty': -1.0,
  'tbObjectivity': 1.0,
  'vsScore': -0.6597},
 {'perspectiveScore': 0.014,
  'tbPolairty': 0.0,
  'tbObjectivity': 0.0,
  'vsScore': 0.0},
 {'perspectiveScore': 0.053,
  'tbPolairty': 0.0,
  'tbObjectivity': 0.0,
  'vsScore': 0.0},
 {'perspectiveScore': 0.046,
  'tbPolairty': 0.0,
  'tbObjectivity': 0.0,
  'vsScore': 0.0}]